In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
import time
from datetime import timedelta
from datetime import datetime
from torchsummary import summary
import math
import os
import shutil
import csv

import importlib
importlib.import_module('FLDataset_temp')
from FLDataset_temp import load_dataset, getActualImgs
from utils import averageModels
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Arguments():
    def __init__(self):
        self.images = 3012
        self.clients = 3
        self.rounds = 1001
        self.epochs = 1
        self.local_batches = 20
        self.lr = 0.01
        self.dropout1 = 0.25
        self.dropout2 = 0.5
        self.C = 0.66
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'noniid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = True
        self.save_model = False
        self.save_model_interval = 500
        self.del_runs = False
        self.acc_csv = False
        self.acc_file = '0601_10clients_woBN_ImageFolder.csv'
        # number of classes per client on non iid case 
        self.noniid_classnum = 1
        # transform
        self.transform = transforms.Compose([
                                    transforms.Grayscale(num_output_channels=1),
                                    transforms.Resize((32,32)),
                                    transforms.ToTensor()])
        # number of classes
        self.c_num = 3

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


cuda


In [3]:
# delete files in runs (Tensorboard)
if args.del_runs==True:
    folder = 'runs'
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

In [4]:
# create accuracy csv file
def acc_csv(args, rnd, acc):
    if args.acc_csv==True:
        with open("acc_csv_files/"+args.acc_file, 'a') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([rnd, acc])

In [5]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

print(clients)
print("number of clients : ", len(clients))

[{'hook': <VirtualWorker id:client1 #objects:0>}, {'hook': <VirtualWorker id:client2 #objects:0>}, {'hook': <VirtualWorker id:client3 #objects:0>}]
number of clients :  3


In [6]:
global_train, train_group = load_dataset(args.clients, \
                                                                  args.iid, \
                                                                  args.transform, \
                                                                  args.c_num, \
                                                                  args.noniid_classnum)

In covid non IID: unsorted labels =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [7]:
print(len(global_train))
print(type(global_train))
print(len(train_group))
print(type(train_group))

3012
<class 'torchvision.datasets.folder.ImageFolder'>
3
<class 'dict'>


In [8]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    print("len(client", str(inx), "train set) = ", len(trainset_ind_list))
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['samples'] = len(trainset_ind_list) / args.images

len(client 0 train set) =  1004
len(client 1 train set) =  1004
len(client 2 train set) =  1004


In [9]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
data_path = "./FinalCovid19Dataset/test"
global_test_dataset = datasets.ImageFolder(root=data_path,
                                        transform=args.transform)
# global_test_dataset = CovidDataset('./test.csv', transform=transforms.Compose([Rescale(32), ToTensor()]))
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True, drop_last=True)
print(len(global_test_loader))

37


In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,
                               out_channels = 32,
                               kernel_size = 3,
                               stride = 1)
        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1)
        self.fc1 = nn.Linear(14*14*64, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))        
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, p=args.dropout1)
        x = x.view(-1, 14*14*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=args.dropout2)
        x = self.fc2(x)
        return F.softmax(x)

In [11]:
def ClientUpdate(args, device, client):
    client['model'].train()
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data)
            loss = client['criterion'](output, target)
            loss.backward()
            
            client['optim'].step()
            
            
            if batch_idx % args.log_interval == 0 or batch_idx==len(client['trainset'])-1:
                print('Model [{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, (batch_idx+1) * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * (batch_idx+1) / len(client['trainset']), loss.item()/args.log_interval))
                

In [12]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
#             data = d['image']
#             target = d['label']
            data, target = data.to(device), target.to(device)
            if(str(device)=='cuda'):
                model.cuda()
            output = model(data.float())
#             test_loss += F.nll_loss(output, target.squeeze(1), reduction='sum').item() # sum up batch loss
            loss_fn = nn.CrossEntropyLoss(reduction='sum')
            test_loss += loss_fn(output, target).item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [13]:
writer = SummaryWriter()

In [14]:
torch.manual_seed(args.torch_seed)
global_model = Net().to(device)
summary(global_model, (1, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 30, 30]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
            Linear-3                  [-1, 128]       1,605,760
            Linear-4                    [-1, 3]             387
Total params: 1,624,963
Trainable params: 1,624,963
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.60
Params size (MB): 6.20
Estimated Total Size (MB): 6.81
----------------------------------------------------------------


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:414: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  response = command_method(*args_, **kwargs_)


In [ ]:
# training
for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)
    client['criterion'] = nn.CrossEntropyLoss(reduction='mean')
    
# start training model
training_start_time = time.time()
for fed_round in range(args.rounds):
    print("")
    print("===================================================================")
    print("[round] = ", fed_round+1, "/", args.rounds)
    print("===================================================================")
    
    round_train_start_time = time.time()
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))    
    
    # number of selected clients
    m = int(max(math.ceil(args.C * args.clients), 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
#     np.random.seed(fed_round)
#     active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
#     active_clients = [clients[i] for i in active_clients_inds]
    active_clients = selected_clients
    
    # Training 
    client_cnt = 0
    for client in active_clients:
        print("* [client count] = ", client_cnt+1 , "/", len(active_clients))
        client_train_start_time = time.time()
        ClientUpdate(args, device, client)
        client_cnt += 1
        client_train_time = round(time.time()-client_train_start_time)
        print("* [client_train_time] = ", str(timedelta(seconds=(client_train_time))))
        print("---------------------------------------------------------------")
    
    # Averaging 
#     print("active clients: ", active_clients)
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    client_test_start_time = time.time()
    acc = test(args, global_model, device, global_test_loader, 'Global')
    client_test_time = round(time.time()-client_test_start_time)
    print("* [global_test_time] = ", str(timedelta(seconds=(client_test_time))))
    
    writer.add_scalar("Accuracy/train", acc, fed_round)
    writer.flush()
    acc_csv(args, fed_round, acc)
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
    # training time per round
    total_train_time = round(time.time()-training_start_time)
    round_train_time = round(time.time()-round_train_start_time)
    print("** [total train time]: ", str(timedelta(seconds=total_train_time)))
    print("** [round train time]: ", str(timedelta(seconds=round_train_time)))
    
    if (args.save_model and fed_round%args.save_model_interval==0 and fed_round!=0):
        now = datetime.now() 
        date = now.strftime("%Y_%m_%d_%H%M")
        torch.save(global_model.state_dict(), date + "_FedAvg_with_DP_round_" + str(fed_round) + ".pth")
        print("model saved : "+ date +"_FedAvg_with_DP_round_" + str(fed_round) + "10clients.pth")



[round] =  1 / 1001
* [client count] =  1 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.109704
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.095912
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.067434
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.060093
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.057848
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055784
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.110487
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.101647
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.083572
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.063393
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.058037
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055970
* [client_train_time] =  0:00:09
-----------------------------------------

Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.056093
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055506
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055525
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055412
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055233
* [client_train_time] =  0:00:03
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.087856
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055738
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055450
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055343
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055326
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055433
* [client_train_time] =  0:00:09
---------------------------------------------------------------

Test set: Average loss for Global model: 1.1005, Accuracy: 272/753 (36%)

* [global_

Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055200
* [client_train_time] =  0:00:03
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.146603
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.056756
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055604
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055501
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055302
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055253
* [client_train_time] =  0:00:09
---------------------------------------------------------------

Test set: Average loss for Global model: 1.0854, Accuracy: 295/753 (39%)

* [global_test_time] =  0:00:06
** [total train time]:  0:04:34
** [round train time]:  0:00:18

[round] =  14 / 1001
* [client count] =  1 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.112353
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.0

Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055952
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055978
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055281
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055281
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055181
* [client_train_time] =  0:00:09
---------------------------------------------------------------

Test set: Average loss for Global model: 1.0641, Accuracy: 318/753 (42%)

* [global_test_time] =  0:00:06
** [total train time]:  0:06:44
** [round train time]:  0:00:17

[round] =  20 / 1001
* [client count] =  1 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.145749
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055536
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055639
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055215
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055232
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss

Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055166
* [client_train_time] =  0:00:09
---------------------------------------------------------------

Test set: Average loss for Global model: 1.0490, Accuracy: 340/753 (45%)

* [global_test_time] =  0:00:06
** [total train time]:  0:08:39
** [round train time]:  0:00:17

[round] =  26 / 1001
* [client count] =  1 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.077408
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055354
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055305
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055249
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055200
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055359
* [client_train_time] =  0:00:09
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.147407
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.0

...

Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055147
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055146
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055167
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055378
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055146
* [client_train_time] =  0:00:03
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client2] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.068102
Model [client2] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055235
Model [client2] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055225
Model [client2] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055185
Model [client2] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055171
Model [client2] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055158
* [client_train_time] =  0:00:09
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9285, Accuracy: 440/753 (58%)

* [global_

Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055163
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.059220
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055199
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055187
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055150
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055145
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:03
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9220, Accuracy: 445/753 (59%)

* [global_test_time] =  0:00:06
** [total train time]:  0:59:50
** [round train time]:  0:00:21

[round] =  169 / 1001
* [client count] =  1 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.058149
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.

Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055156
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055147
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055152
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055145
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:03
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9098, Accuracy: 451/753 (60%)

* [global_test_time] =  0:00:06
** [total train time]:  1:01:49
** [round train time]:  0:00:21

[round] =  175 / 1001
* [client count] =  1 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.062737
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055822
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055296
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055145
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055146
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Los

Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055145
* [client_train_time] =  0:00:12
---------------------------------------------------------------

Test set: Average loss for Global model: 0.9121, Accuracy: 449/753 (60%)

* [global_test_time] =  0:00:06
** [total train time]:  1:04:04
** [round train time]:  0:00:21

[round] =  181 / 1001
* [client count] =  1 / 2
Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.086619
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055296
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055164
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055159
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055338
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055184
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.060389
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.

Model [client3] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.081364
Model [client3] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055145
Model [client3] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055147
Model [client3] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055186
Model [client3] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055145
Model [client3] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055147
* [client_train_time] =  0:00:12
---------------------------------------------------------------
* [client count] =  2 / 2
Model [client1] Train Epoch: 1 [20/1020 (2%)]	Loss: 0.115528
Model [client1] Train Epoch: 1 [220/1020 (22%)]	Loss: 0.055153
Model [client1] Train Epoch: 1 [420/1020 (41%)]	Loss: 0.055150
Model [client1] Train Epoch: 1 [620/1020 (61%)]	Loss: 0.055187
Model [client1] Train Epoch: 1 [820/1020 (80%)]	Loss: 0.055151
Model [client1] Train Epoch: 1 [1020/1020 (100%)]	Loss: 0.055146
* [client_train_time] =  0:00:03
---------------------------------------------------------------

Test set: Average loss 

In [ ]:
# tensorboard open 
# tensorboard --logdir=/home/citi302/Desktop/Codefolder/FL_DP_covid/runs